<a href="https://colab.research.google.com/github/Dipesh0987/Concepts-and-Technologies-of-AI/blob/main/DipeshKishorChhetri_worksheet10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/SMSSpamCollection", index_col=0)
length = len(df['text'])

AttributeError: module 'pandas' has no attribute 'to_csv'

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
# contain list of words that will be used for training -> final words after cleaning
for i in range(0,length):
  # re is used to remove punctuations
  text = re.sub('[^a-zA-Z]',' ',df['text'][i]) # replaces the punctuation with space
  # converting to lowercase
  text=text.lower()
  # stemming
  text=text.split()
  ps = PorterStemmer()
  all_stopwords=stopwords.words('english')
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  text = ' '.join(text)
  corpus.append(text)